# HTMDEC DMS API Example

This example demonstrates how to use the new HTMDEC DMS `/form` and `/entry` REST API endpoints to search for BIRDSHOT data which can be converted to a Pandas dataframe for analysis and visualization. 

The `birdshot.py` module implements some example utility functions for querying, analyzing, and visualizing BIRDSHOT data.

### Note:
During the ingest process, form data collected by Contextualize in the Campaign 1 structure was converted to the [Campaign 2 structure](https://docs.google.com/document/d/1FpiXwLQi8QAOuLB0Xr80V14qGfqtyCpT/edit). This means that:
* Sample IDs were converted to the Campaign 2 structure 
* Campaign 1 "child" sample numbers (T*nn*) have been replaced with ordinal letters (a, b, c, ...)


### Using the REST API

The [Girder Client](https://girder.readthedocs.io/en/latest/python-client.html) can be used to query the REST endpoints directly. When running via the DMS, the API URL and user token required for access are available in the current environment.


In [10]:
from girder_client import GirderClient
import os

client = GirderClient(apiUrl=os.environ['GIRDER_API_URL'])
client.token = os.environ['GIRDER_TOKEN']

To query VAM data from iteration 1 (AAA) across all forms, use the `/entry/search` endpoint to query the Sample ID. Data is returned in JSON:

In [8]:
iteration = 'BBB'
raw_data = client.get(
        'entry/search', parameters={'query': f'^{iteration}.._VAM-.', 'limit': 1000}
)
raw_data[0]["data"]

{'Forging': {'Ingot Condition': {'Soak Time': 0, 'Temperature': 25},
  'Ingot Dimensions After': {'Length': 63.57,
   'Thickness': 4.92,
   'Thickness Reduction': -61.13744075829384,
   'Width': 15},
  'Ingot Dimensions Before': {'Length': 38.83,
   'Thickness': 12.66,
   'Width': 14.57},
  'Maximum Load': [{'Maximum Load Step': 0}],
  'Press Temperature': 25,
  'Process Overview': {'Completed By': 'unknown',
   'Finish Date': '2024-04-30',
   'Start Date': '2024-04-01',
   'Time Spent': '5:00'}},
 'Homogenization': {'Process Overview': {'Completed By': 'unknown',
   'Finish Date': '2024-04-30',
   'Start Date': '2024-04-01',
   'Time Spent': '5:00'},
  'Purging Sequence Pressure': {'1': 0, '2': 0, '3': 0, '4': 0},
  'Thermal Conditions': {'Atmosphere': 'Ar',
   'Cooling Rate': 'FC',
   'Duration': 24,
   'Pressure': 0,
   'Temperature': 1150}},
 'Notes': '',
 'sampleId': 'BBB01_VAM-A',
 'suffix': 'Syn',
 'targetPath': 'BBB/VAM-A/BBB01/Syn'}

Use the `/form` and `entry` endpoints to query data from a specific form:

In [3]:
form_name = 'tensile-details.json'
form = client.get('form', parameters={'entryFileName': form_name, 'limit': 1000})
form[0]['_id']

tensile_data = client.get('entry', parameters={'formId': form[0]['_id'], 'limit': 1000})
tensile_data[0]["data"]

{'Completed By': 'Daniel & Wenle',
 'End Date': '2022-12-13',
 'File Upload': {'file': '',
  'sampleId': 'AAB06_VAM-A_Tensile_a',
  'targetPath': 'AAB/VAM-A/AAB06/Tensile/a'},
 'Modulus Check': {'Force Rate': 10, 'Max Force': 686, 'Max Stress': 225},
 'Notes': '',
 'Results': {'Elastic Modulus': 252,
  'Elongation': 28,
  'Maximum ∂2σ/∂ε2': -1529,
  'UTS/YS Ratio': '1.6794871794871795',
  'Ultimate Tensile Strength': 917,
  'Yield Strength': 546},
 'Sample Dimensions': {'Area': '3.0488',
  'Gauge Length': 8,
  'Thickness': 1.03,
  'Width': 2.96},
 'Start Date': '2022-11-15',
 'Time Spent': '0:04',
 'sampleId': 'AAB06_VAM-A_Tensile_a',
 'suffix': 'Tensile',
 'targetPath': 'AAB/VAM-A/AAB06/Tensile/a'}

### Using the `birdshot` Module

The `birdshot.py` module implements a few helper functions to query the REST API and convert data into a single Pandas dataframe for analysis:

In [13]:
import birdshot

The `query()` method takes the iteration identifier as an argument and returns a dataframe of results from multiple characterization methods. The example dataframe is intended to reproduce the information available in the Summary Synthesis Results (for example see [HTMDEC AAB Summary Synthesis Results](https://docs.google.com/spreadsheets/d/15cdImpOComsvUpAIq20_nyff65WVzN_q/)).

In [17]:
df = birdshot.query("BBB")
df

KeyError: "Only a column name can be used for the key in a dtype mappings argument. 'UTS/YS Ratio.a' not found in columns."